# 제품 이상여부 판별 프로젝트


## 1. 데이터 불러오기


### 필수 라이브러리


In [20]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

### 데이터 읽어오기


In [21]:
RANDOM_STATE = 110

train_data = pd.read_csv("C:/Users/KimDongyoung/Desktop/git_LGaimers5/Lg_aimers5/data/train_data.csv")
test_data = pd.read_csv("C:/Users/KimDongyoung/Desktop/git_LGaimers5/Lg_aimers5/data/test_data.csv")

In [22]:
train_data.info()
print('---')
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Columns: 119 entries, Model.Suffix to Dispenser_num
dtypes: float64(58), int64(51), object(10)
memory usage: 36.8+ MB
---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17361 entries, 0 to 17360
Columns: 120 entries, Set ID to Dispenser_num
dtypes: float64(94), int64(16), object(10)
memory usage: 15.9+ MB


공통 공정 변수 처리

In [23]:
train_data['Dispenser_num'].value_counts()

Dispenser_num
#1    25011
#2    15461
#0       34
Name: count, dtype: int64

In [24]:
# Dispenser_num 값에 따라 새로운 변수 생성
train_data['Dispenser_1'] = train_data['Dispenser_num'].apply(lambda x: 1 if x == '#1' else 0)
train_data['Dispenser_2'] = train_data['Dispenser_num'].apply(lambda x: 1 if x == '#2' else 0)

test_data['Dispenser_1'] = test_data['Dispenser_num'].apply(lambda x: 1 if x == '#1' else 0)
test_data['Dispenser_2'] = test_data['Dispenser_num'].apply(lambda x: 1 if x == '#2' else 0)

# 결과 확인
print(train_data[['Dispenser_num', 'Dispenser_1', 'Dispenser_2']].head(5))

# 불필요한 변수 제거
train_data.drop(['Dispenser_num'], axis=1, inplace=True)
test_data.drop(['Dispenser_num'], axis=1, inplace=True)

  Dispenser_num  Dispenser_1  Dispenser_2
0            #1            1            0
1            #1            1            0
2            #2            0            1
3            #2            0            1
4            #1            1            0


In [25]:
# 타겟 변수를 0과 1로 변환
train_data['target_binary'] = train_data['target'].apply(lambda x: 1 if x == 'AbNormal' else 0)

# Workorder 변수의 값에 대한 타겟 변수 비율 계산
workorder_target_ratio = train_data.groupby('Workorder')['target_binary'].mean()

# 파생 변수 생성 함수
def create_derived_variable(row, ratio_dict, threshold):
    return 1 if ratio_dict.get(row['Workorder'], 0) >= threshold else 0

# 파생 변수 생성
train_data['Workorder_0.9'] = train_data.apply(create_derived_variable, axis=1, ratio_dict=workorder_target_ratio, threshold=0.9)
train_data['Workorder_0.7'] = train_data.apply(create_derived_variable, axis=1, ratio_dict=workorder_target_ratio, threshold=0.7)
train_data['Workorder_0.5'] = train_data.apply(create_derived_variable, axis=1, ratio_dict=workorder_target_ratio, threshold=0.5)

test_data['Workorder_0.9'] = test_data.apply(create_derived_variable, axis=1, ratio_dict=workorder_target_ratio, threshold=0.9)
test_data['Workorder_0.7'] = test_data.apply(create_derived_variable, axis=1, ratio_dict=workorder_target_ratio, threshold=0.7)
test_data['Workorder_0.5'] = test_data.apply(create_derived_variable, axis=1, ratio_dict=workorder_target_ratio, threshold=0.5)


# 불필요한 변수 제거
train_data.drop(['target_binary'], axis=1, inplace=True)

# train_data.drop(['Workorder', 'target_binary'], axis=1, inplace=True)
# test_data.drop(['Workorder'], axis=1, inplace=True)

In [26]:
train_data.columns.to_list()

['Model.Suffix',
 'Workorder',
 'CURE END POSITION X Collect Result_Dam',
 'CURE END POSITION Z Collect Result_Dam',
 'CURE END POSITION Θ Collect Result_Dam',
 'CURE SPEED Collect Result_Dam',
 'CURE START POSITION X Collect Result_Dam',
 'CURE START POSITION Θ Collect Result_Dam',
 'DISCHARGED SPEED OF RESIN Collect Result_Dam',
 'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam',
 'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam',
 'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam',
 'Dispense Volume(Stage1) Collect Result_Dam',
 'Dispense Volume(Stage2) Collect Result_Dam',
 'Dispense Volume(Stage3) Collect Result_Dam',
 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
 'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam',
 'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam',
 'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam',
 'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam',
 'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_D

In [27]:
train_data['Model.Suffix'].value_counts()

Model.Suffix
AJX75334501    33820
AJX75334502     3390
AJX75334505     2635
AJX75334507      310
AJX75334503      162
AJX75334506      129
AJX75334508       60
Name: count, dtype: int64

---

개별 공정별 데이터 전처리

## Dam

In [28]:
# '_Dam'를 포함하는 열 이름 필터링
Process_Desc_col = train_data.filter(like='_Dam').columns

# 필터링된 열 이름 출력
print("<Dam 공정 관련 변수>")
for col in Process_Desc_col:
    print(col)

<Dam 공정 관련 변수>
CURE END POSITION X Collect Result_Dam
CURE END POSITION Z Collect Result_Dam
CURE END POSITION Θ Collect Result_Dam
CURE SPEED Collect Result_Dam
CURE START POSITION X Collect Result_Dam
CURE START POSITION Θ Collect Result_Dam
DISCHARGED SPEED OF RESIN Collect Result_Dam
DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam
DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam
DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam
Dispense Volume(Stage1) Collect Result_Dam
Dispense Volume(Stage2) Collect Result_Dam
Dispense Volume(Stage3) Collect Result_Dam
HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam
HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam
HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam
HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam
HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam
HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam
HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam
HEAD NORMAL COORDINATE Z AXIS(

In [29]:
import numpy as np

# 시작 위치와 끝 위치 열 이름
start_x_col = 'CURE START POSITION X Collect Result_Dam'
start_z_col = 33.5
end_x_col = 'CURE END POSITION X Collect Result_Dam'
end_z_col = 'CURE END POSITION Z Collect Result_Dam'

# 시작 위치와 끝 위치 사이의 거리 계산
train_data['CURE_DISTANCE_Dam'] = np.sqrt(
    (train_data[end_x_col] - train_data[start_x_col]) ** 2 +
    (train_data[end_z_col] - start_z_col) ** 2
)

test_data['CURE_DISTANCE_Dam'] = np.sqrt(
    (train_data[end_x_col] - train_data[start_x_col]) ** 2 +
    (train_data[end_z_col] - start_z_col) ** 2
)

In [30]:
# 변수 제거
train_data.drop(columns=[
    'CURE END POSITION X Collect Result_Dam'
    , 'CURE END POSITION Z Collect Result_Dam'
    , 'CURE END POSITION Θ Collect Result_Dam'
    , 'CURE START POSITION X Collect Result_Dam'
    , 'CURE START POSITION Θ Collect Result_Dam'
], inplace=True)

test_data.drop(columns=[
    'CURE END POSITION X Collect Result_Dam'
    , 'CURE END POSITION Z Collect Result_Dam'
    , 'CURE END POSITION Θ Collect Result_Dam'
    , 'CURE START POSITION X Collect Result_Dam'
    , 'CURE START POSITION Θ Collect Result_Dam'
], inplace=True)

In [31]:
train_data['CURE_Time_Dam'] = train_data['CURE_DISTANCE_Dam'] / train_data['CURE SPEED Collect Result_Dam']
test_data['CURE_Time_Dam'] = test_data['CURE_DISTANCE_Dam'] / test_data['CURE SPEED Collect Result_Dam']

In [32]:
train_data.drop(columns=[
    'CURE_DISTANCE_Dam'
    , 'CURE SPEED Collect Result_Dam'
], inplace=True)

test_data.drop(columns=[
    'CURE_DISTANCE_Dam'
    , 'CURE SPEED Collect Result_Dam'
], inplace=True)

In [33]:
import numpy as np

# 각 스테이지의 좌표 열 정의
stage1_cols = ['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
               'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam',
               'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam']

stage2_cols = ['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam',
               'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam',
               'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam']

stage3_cols = ['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam',
               'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam',
               'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam']

# 거리 계산 함수
def calculate_distances(data):
    data['DISTANCE_STAGE1_STAGE2_Dam'] = np.sqrt(
        (data[stage2_cols[0]] - data[stage1_cols[0]]) ** 2 +
        (data[stage2_cols[1]] - data[stage1_cols[1]]) ** 2 +
        (data[stage2_cols[2]] - data[stage1_cols[2]]) ** 2
    )

    data['DISTANCE_STAGE2_STAGE3_Dam'] = np.sqrt(
        (data[stage3_cols[0]] - data[stage2_cols[0]]) ** 2 +
        (data[stage3_cols[1]] - data[stage2_cols[1]]) ** 2 +
        (data[stage3_cols[2]] - data[stage2_cols[2]]) ** 2
    )

    data['DISTANCE_STAGE1_STAGE3_Dam'] = np.sqrt(
        (data[stage3_cols[0]] - data[stage1_cols[0]]) ** 2 +
        (data[stage3_cols[1]] - data[stage1_cols[1]]) ** 2 +
        (data[stage3_cols[2]] - data[stage1_cols[2]]) ** 2
    )

    return data

# train_data에 적용
train_data = calculate_distances(train_data)

# test_data에 적용
test_data = calculate_distances(test_data)

In [34]:
import numpy as np

# 필요한 열 이름
stage1_stage2_col = 'DISTANCE_STAGE1_STAGE2_Dam'
stage2_stage3_col = 'DISTANCE_STAGE2_STAGE3_Dam'
stage1_stage3_col = 'DISTANCE_STAGE1_STAGE3_Dam'

# 삼각형의 넓이와 높이를 계산하는 함수
def calculate_triangle_features(data):
    a = data[stage1_stage2_col]
    b = data[stage2_stage3_col]
    c = data[stage1_stage3_col]

    # 헤론의 공식에 따른 삼각형의 넓이 계산
    s = (a + b + c) / 2
    area = np.sqrt(s * (s - a) * (s - b) * (s - c))

    # 높이 계산 (밑변을 c로 가정)
    height = (2 * area) / c

    # 결과를 새로운 열에 저장
    data['DISTANCE_TRIANGLE_area_Dam'] = area
    data['DISTANCE_TRIANGLE_height_Dam'] = height

    return data

# train_data에 적용
train_data = calculate_triangle_features(train_data)

# test_data에 적용
test_data = calculate_triangle_features(test_data)

In [35]:
import pandas as pd

# '_Dam'를 포함하는 열 이름 필터링 함수
def filter_dam_columns(data):
    return data.filter(like='_Dam').columns

# 스테이지별로 파생변수 생성 함수
def create_stage_variables(data, columns):
    stages = ['Stage1', 'Stage2', 'Stage3']
    for stage in stages:
        # Filter stage-specific columns
        stage_cols = [col for col in columns if stage in col]
        
        # Ensure that only numeric columns are summed
        stage_numeric_cols = data[stage_cols].select_dtypes(include=['number'])

        # Create new derived column for the sum of the numeric stage columns
        new_col_name = f'{stage}_Total_Distance_Speed_Collect_Result_Dam'
        data[new_col_name] = stage_numeric_cols.sum(axis=1)
        
        # 이전 변수 삭제
        data.drop(columns=stage_cols, inplace=True)
    
    return data

# train_data에 적용
train_dam_columns = filter_dam_columns(train_data)
train_data = create_stage_variables(train_data, train_dam_columns)

# test_data에 적용
test_dam_columns = filter_dam_columns(test_data)
test_data = create_stage_variables(test_data, test_dam_columns)

In [36]:
# 새로운 파생변수 생성 함수
def create_total_thickness_dam(data):
    data['Total_THICKNESS_Collect_Result_Dam'] = (
        data['THICKNESS 1 Collect Result_Dam']**2 
        + data['THICKNESS 2 Collect Result_Dam']**2 
        + data['THICKNESS 3 Collect Result_Dam']**2
    )
    # 기존 변수 삭제
    data.drop(columns=[
        'THICKNESS 1 Collect Result_Dam',
        'THICKNESS 2 Collect Result_Dam',
        'THICKNESS 3 Collect Result_Dam'
    ], inplace=True)
    return data

train_data = create_total_thickness_dam(train_data)
test_data = create_total_thickness_dam(test_data)

In [37]:
train_data['Total_THICKNESS_Collect_Result_Dam'].value_counts()

Total_THICKNESS_Collect_Result_Dam
0.000000    35419
0.014726     1649
0.003704     1197
0.002197      964
0.050926      884
0.000389      150
0.000637      125
0.001970      118
Name: count, dtype: int64

In [38]:
# 삭제할 열 이름 정의
columns_to_drop = [
    'DISTANCE_STAGE1_STAGE2_Dam'
    , 'DISTANCE_STAGE2_STAGE3_Dam'
    # , 'DISTANCE_STAGE1_STAGE3_Dam'
]

# train_data에서 열 삭제
train_data = train_data.drop(columns=columns_to_drop)

# test_data에서 열 삭제
test_data = test_data.drop(columns=columns_to_drop)

In [39]:
# '_Dam'를 포함하는 열 이름 필터링
Process_Desc_col = train_data.filter(like='_Dam').columns

# 필터링된 열 이름 출력
print("<Dam 공정 관련 변수>")
for col in Process_Desc_col:
    print(col)

<Dam 공정 관련 변수>
DISCHARGED SPEED OF RESIN Collect Result_Dam
Head Clean Position Z Collect Result_Dam
Head Purge Position Z Collect Result_Dam
Head Zero Position Y Collect Result_Dam
Head Zero Position Z Collect Result_Dam
Machine Tact time Collect Result_Dam
PalletID Collect Result_Dam
Production Qty Collect Result_Dam
Receip No Collect Result_Dam
WorkMode Collect Result_Dam
CURE_Time_Dam
DISTANCE_STAGE1_STAGE3_Dam
DISTANCE_TRIANGLE_area_Dam
DISTANCE_TRIANGLE_height_Dam
Stage1_Total_Distance_Speed_Collect_Result_Dam
Stage2_Total_Distance_Speed_Collect_Result_Dam
Stage3_Total_Distance_Speed_Collect_Result_Dam
Total_THICKNESS_Collect_Result_Dam


In [40]:
train_data['All_Total_Distance_Speed_Collect_Result_Dam'] = train_data['Stage1_Total_Distance_Speed_Collect_Result_Dam'] + train_data['Stage2_Total_Distance_Speed_Collect_Result_Dam'] + train_data['Stage3_Total_Distance_Speed_Collect_Result_Dam']
test_data['All_Total_Distance_Speed_Collect_Result_Dam'] = test_data['Stage1_Total_Distance_Speed_Collect_Result_Dam'] + test_data['Stage2_Total_Distance_Speed_Collect_Result_Dam'] + test_data['Stage3_Total_Distance_Speed_Collect_Result_Dam']

train_data['Distance_frac_time_Dam'] = train_data['All_Total_Distance_Speed_Collect_Result_Dam'] / train_data['Machine Tact time Collect Result_Dam']
test_data['Distance_frac_time_Dam'] = test_data['All_Total_Distance_Speed_Collect_Result_Dam'] / test_data['Machine Tact time Collect Result_Dam']

# 삭제할 열 이름 정의
columns_to_drop = [
    # 'Machine Tact time Collect Result_Dam'
    # , 'Stage1_Total_Distance_Speed_Collect_Result_Dam'
    # , 'Stage2_Total_Distance_Speed_Collect_Result_Dam'
    # , 'Stage3_Total_Distance_Speed_Collect_Result_Dam'
     'All_Total_Distance_Speed_Collect_Result_Dam'
]

# train_data에서 열 삭제
train_data = train_data.drop(columns=columns_to_drop)

# test_data에서 열 삭제
test_data = test_data.drop(columns=columns_to_drop)

In [41]:
# '_Dam'를 포함하는 열 이름 필터링
Process_Desc_col = train_data.filter(like='_Dam').columns

# 필터링된 열 이름 출력
print("<Dam 공정 관련 변수>")
for col in Process_Desc_col:
    print(col)

<Dam 공정 관련 변수>
DISCHARGED SPEED OF RESIN Collect Result_Dam
Head Clean Position Z Collect Result_Dam
Head Purge Position Z Collect Result_Dam
Head Zero Position Y Collect Result_Dam
Head Zero Position Z Collect Result_Dam
Machine Tact time Collect Result_Dam
PalletID Collect Result_Dam
Production Qty Collect Result_Dam
Receip No Collect Result_Dam
WorkMode Collect Result_Dam
CURE_Time_Dam
DISTANCE_STAGE1_STAGE3_Dam
DISTANCE_TRIANGLE_area_Dam
DISTANCE_TRIANGLE_height_Dam
Stage1_Total_Distance_Speed_Collect_Result_Dam
Stage2_Total_Distance_Speed_Collect_Result_Dam
Stage3_Total_Distance_Speed_Collect_Result_Dam
Total_THICKNESS_Collect_Result_Dam
Distance_frac_time_Dam


## AutoClave

In [42]:
# '_AutoClave'를 포함하는 열 이름 필터링
Process_Desc_col = train_data.filter(like='_AutoClave').columns

# 필터링된 열 이름 출력
print("<AutoClave 공정 관련 변수>")
for col in Process_Desc_col:
    print(col)

<AutoClave 공정 관련 변수>
1st Pressure Collect Result_AutoClave
1st Pressure 1st Pressure Unit Time_AutoClave
2nd Pressure Collect Result_AutoClave
2nd Pressure Unit Time_AutoClave
3rd Pressure Collect Result_AutoClave
3rd Pressure Unit Time_AutoClave
Chamber Temp. Collect Result_AutoClave
Chamber Temp. Unit Time_AutoClave
Chamber Temp. Judge Value_AutoClave
GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave
GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave


In [43]:
# 변수명 변경
train_data = train_data.rename(columns={'1st Pressure 1st Pressure Unit Time_AutoClave': '1st Pressure Unit Time_AutoClave'})
test_data = test_data.rename(columns={'1st Pressure 1st Pressure Unit Time_AutoClave': '1st Pressure Unit Time_AutoClave'})

train_data = train_data.rename(columns={'GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave': 'GMES_ORIGIN_INSP_JUDGE_CODE_AutoClave'})
test_data = test_data.rename(columns={'GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave': 'GMES_ORIGIN_INSP_JUDGE_CODE_AutoClave'})

In [44]:
# 파생변수 생성
train_data['1st_Pressure_x_AutoClave'] = train_data['1st Pressure Collect Result_AutoClave'] * train_data['1st Pressure Unit Time_AutoClave'] 
test_data['1st_Pressure_x_AutoClave'] = test_data['1st Pressure Collect Result_AutoClave'] * test_data['1st Pressure Unit Time_AutoClave'] 

train_data['2nd_Pressure_x_AutoClave'] = train_data['2nd Pressure Collect Result_AutoClave'] * train_data['2nd Pressure Unit Time_AutoClave'] 
test_data['2nd_Pressure_x_AutoClave'] = test_data['2nd Pressure Collect Result_AutoClave'] * test_data['2nd Pressure Unit Time_AutoClave'] 

train_data['3rd_Pressure_x_AutoClave'] = train_data['3rd Pressure Collect Result_AutoClave'] * train_data['3rd Pressure Unit Time_AutoClave'] 
test_data['3rd_Pressure_x_AutoClave'] = test_data['3rd Pressure Collect Result_AutoClave'] * test_data['3rd Pressure Unit Time_AutoClave'] 

train_data['All_Pressure_x_AutoClave'] = train_data['1st_Pressure_x_AutoClave'] + train_data['2nd_Pressure_x_AutoClave'] + train_data['3rd_Pressure_x_AutoClave']
test_data['All_Pressure_x_AutoClave'] = test_data['1st_Pressure_x_AutoClave'] + test_data['2nd_Pressure_x_AutoClave'] + test_data['3rd_Pressure_x_AutoClave']

train_data['All_Pressure_avg_AutoClave'] = train_data['All_Pressure_x_AutoClave'] / train_data['Chamber Temp. Unit Time_AutoClave']
test_data['All_Pressure_avg_AutoClave'] = test_data['All_Pressure_x_AutoClave'] / test_data['Chamber Temp. Unit Time_AutoClave']

train_data['Chamber_Temp_x_AutoClave'] = train_data['Chamber Temp. Collect Result_AutoClave'] * train_data['Chamber Temp. Unit Time_AutoClave']
test_data['Chamber_Temp_x_AutoClave'] = test_data['Chamber Temp. Collect Result_AutoClave'] * test_data['Chamber Temp. Unit Time_AutoClave']

train_data['Chamber_Temp_ok_AutoClave'] = train_data['Chamber Temp. Judge Value_AutoClave'].apply(lambda x: 1 if x == 'OK' else 0)
test_data['Chamber_Temp_ok_AutoClave'] = test_data['Chamber Temp. Judge Value_AutoClave'].apply(lambda x: 1 if x == 'OK' else 0)

train_data['GMES_ORIGIN_INSP_JUDGE_CODE_ok_AutoClave'] = train_data['GMES_ORIGIN_INSP_JUDGE_CODE_AutoClave'].apply(lambda x: 1 if x == 'OK' else 0)
test_data['GMES_ORIGIN_INSP_JUDGE_CODE_ok_AutoClave'] = test_data['GMES_ORIGIN_INSP_JUDGE_CODE_AutoClave'].apply(lambda x: 1 if x == 'OK' else 0)


In [45]:
# 변수 제거
train_data.drop(columns=[
    # 'Chamber Temp. Collect Result_AutoClave'
    #  'Chamber Temp. Unit Time_AutoClave'
     'Chamber Temp. Judge Value_AutoClave'
    , 'GMES_ORIGIN_INSP_JUDGE_CODE_AutoClave'
    , 'GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave'
    # , '1st Pressure Collect Result_AutoClave'
    , '1st Pressure Unit Time_AutoClave'
    # , '2nd Pressure Collect Result_AutoClave'
    , '2nd Pressure Unit Time_AutoClave'
    # , '3rd Pressure Collect Result_AutoClave'
    , '3rd Pressure Unit Time_AutoClave'], inplace=True)

test_data.drop(columns=[
    # 'Chamber Temp. Collect Result_AutoClave'
    #  'Chamber Temp. Unit Time_AutoClave'
     'Chamber Temp. Judge Value_AutoClave'
    , 'GMES_ORIGIN_INSP_JUDGE_CODE_AutoClave'
    , 'GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave'
    # , '1st Pressure Collect Result_AutoClave'
    , '1st Pressure Unit Time_AutoClave'
    # , '2nd Pressure Collect Result_AutoClave'
    , '2nd Pressure Unit Time_AutoClave'
    # , '3rd Pressure Collect Result_AutoClave'
    , '3rd Pressure Unit Time_AutoClave'], inplace=True)

In [46]:
# '_AutoClave'를 포함하는 열 이름 필터링
Process_Desc_col = train_data.filter(like='_AutoClave').columns

# 필터링된 열 이름 출력
print("<AutoClave 공정 관련 변수>")
for col in Process_Desc_col:
    print(col)

<AutoClave 공정 관련 변수>
1st Pressure Collect Result_AutoClave
2nd Pressure Collect Result_AutoClave
3rd Pressure Collect Result_AutoClave
Chamber Temp. Collect Result_AutoClave
Chamber Temp. Unit Time_AutoClave
1st_Pressure_x_AutoClave
2nd_Pressure_x_AutoClave
3rd_Pressure_x_AutoClave
All_Pressure_x_AutoClave
All_Pressure_avg_AutoClave
Chamber_Temp_x_AutoClave
Chamber_Temp_ok_AutoClave
GMES_ORIGIN_INSP_JUDGE_CODE_ok_AutoClave


## Fill1

In [47]:
import numpy as np

# 각 스테이지의 좌표 열 정의
stage1_cols = ['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
               'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1',
               'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1']

stage2_cols = ['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1',
               'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1',
               'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1']

stage3_cols = ['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1',
               'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1',
               'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1']

# 거리 계산 함수
def calculate_distances(data):
    data['DISTANCE_STAGE1_STAGE2_Fill1'] = np.sqrt(
        (data[stage2_cols[0]] - data[stage1_cols[0]]) ** 2 +
        (data[stage2_cols[1]] - data[stage1_cols[1]]) ** 2 +
        (data[stage2_cols[2]] - data[stage1_cols[2]]) ** 2
    )

    data['DISTANCE_STAGE2_STAGE3_Fill1'] = np.sqrt(
        (data[stage3_cols[0]] - data[stage2_cols[0]]) ** 2 +
        (data[stage3_cols[1]] - data[stage2_cols[1]]) ** 2 +
        (data[stage3_cols[2]] - data[stage2_cols[2]]) ** 2
    )

    data['DISTANCE_STAGE1_STAGE3_Fill1'] = np.sqrt(
        (data[stage3_cols[0]] - data[stage1_cols[0]]) ** 2 +
        (data[stage3_cols[1]] - data[stage1_cols[1]]) ** 2 +
        (data[stage3_cols[2]] - data[stage1_cols[2]]) ** 2
    )

    return data

# train_data에 적용
train_data = calculate_distances(train_data)

# test_data에 적용
test_data = calculate_distances(test_data)

In [48]:
import numpy as np

# 필요한 열 이름
stage1_stage2_col = 'DISTANCE_STAGE1_STAGE2_Fill1'
stage2_stage3_col = 'DISTANCE_STAGE2_STAGE3_Fill1'
stage1_stage3_col = 'DISTANCE_STAGE1_STAGE3_Fill1'

# 삼각형의 넓이와 높이를 계산하는 함수
def calculate_triangle_features(data):
    a = data[stage1_stage2_col]
    b = data[stage2_stage3_col]
    c = data[stage1_stage3_col]

    # 헤론의 공식에 따른 삼각형의 넓이 계산
    s = (a + b + c) / 2
    area = np.sqrt(s * (s - a) * (s - b) * (s - c))

    # 높이 계산 (밑변을 c로 가정)
    height = (2 * area) / c

    # 결과를 새로운 열에 저장
    data['DISTANCE_TRIANGLE_area_Fill1'] = area
    data['DISTANCE_TRIANGLE_height_Fill1'] = height

    return data

# train_data에 적용
train_data = calculate_triangle_features(train_data)

# test_data에 적용
test_data = calculate_triangle_features(test_data)

In [49]:
# 삭제할 열 이름 정의
columns_to_drop = [
    'DISTANCE_STAGE1_STAGE2_Fill1'
    , 'DISTANCE_STAGE2_STAGE3_Fill1'
    , 'DISTANCE_STAGE1_STAGE3_Fill1'
]

# train_data에서 열 삭제
train_data = train_data.drop(columns=columns_to_drop)

# test_data에서 열 삭제
test_data = test_data.drop(columns=columns_to_drop)

In [50]:
# 변수 제거
train_data.drop(columns=[
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'
    , 'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1'
    , 'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1'
    , 'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1'
    , 'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1'
    , 'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1'
    , 'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1'
    , 'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1'
    , 'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1'], inplace=True)

test_data.drop(columns=[
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'
    , 'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1'
    , 'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1'
    , 'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1'
    , 'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1'
    , 'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1'
    , 'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1'
    , 'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1'
    , 'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1'], inplace=True)

In [51]:
# 'WorkMode Collect Result_Fill1' 열의 결측값을 0으로 채우기
train_data['WorkMode Collect Result_Fill1'].fillna(0, inplace=True)
test_data['WorkMode Collect Result_Fill1'].fillna(0, inplace=True)

C:\Users\KimDongyoung\AppData\Local\Temp\ipykernel_30836\4033227230.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data['WorkMode Collect Result_Fill1'].fillna(0, inplace=True)
C:\Users\KimDongyoung\AppData\Local\Temp\ipykernel_30836\4033227230.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values a

In [52]:
# '_Fill1'를 포함하는 열 이름 필터링
Process_Desc_col = train_data.filter(like='_Fill1').columns

# 필터링된 열 이름 출력
print("<Fill1 공정 관련 변수>")
for col in Process_Desc_col:
    print(col)

<Fill1 공정 관련 변수>
DISCHARGED SPEED OF RESIN Collect Result_Fill1
DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1
DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1
DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1
Dispense Volume(Stage1) Collect Result_Fill1
Dispense Volume(Stage2) Collect Result_Fill1
Dispense Volume(Stage3) Collect Result_Fill1
HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1
Head Purge Position Z Collect Result_Fill1
Machine Tact time Collect Result_Fill1
PalletID Collect Result_Fill1
Production Qty Collect Result_Fill1
Receip No Collect Result_Fill1
WorkMode Collect Result_Fill1
DISTANCE_TRIANGLE_area_Fill1
DISTANCE_TRIANGLE_height_Fill1


## Fill2

In [53]:
# '_Fill2'를 포함하는 열 이름 필터링
Process_Desc_col = train_data.filter(like='_Fill2').columns

# 필터링된 열 이름 출력
print("<Fill2 공정 관련 변수>")
for col in Process_Desc_col:
    print(col)

<Fill2 공정 관련 변수>
CURE END POSITION X Collect Result_Fill2
CURE END POSITION Z Collect Result_Fill2
CURE SPEED Collect Result_Fill2
CURE STANDBY POSITION Z Collect Result_Fill2
CURE START POSITION X Collect Result_Fill2
CURE START POSITION Z Collect Result_Fill2
HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2
HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill2
HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2
HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2
HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill2
HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2
HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill2
HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2
HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2
HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill2
Head Purge Position Z Collect Result_Fill2
Machine Tact time Collect Result_Fill2
PalletID Collect Result_Fill2
Production Qty Collect Result_Fill2
Rec

In [54]:
import numpy as np

# 시작 위치와 끝 위치 열 이름
start_x_col = 'CURE START POSITION X Collect Result_Fill2'
start_z_col = 'CURE START POSITION Z Collect Result_Fill2'
end_x_col = 'CURE END POSITION X Collect Result_Fill2'
end_z_col = 'CURE END POSITION Z Collect Result_Fill2'

# 시작 위치와 끝 위치 사이의 거리 계산
train_data['CURE_DISTANCE_Fill2'] = np.sqrt(
    (train_data[end_x_col] - train_data[start_x_col]) ** 2 +
    (train_data[end_z_col] - train_data[start_z_col]) ** 2
)

test_data['CURE_DISTANCE_Fill2'] = np.sqrt(
    (train_data[end_x_col] - train_data[start_x_col]) ** 2 +
    (train_data[end_z_col] - train_data[start_z_col]) ** 2
)

In [55]:
train_data['CURE_Time_Fill2']  = train_data['CURE_DISTANCE_Fill2'] / train_data['CURE SPEED Collect Result_Fill2']
test_data['CURE_Time_Fill2']  = test_data['CURE_DISTANCE_Fill2'] / test_data['CURE SPEED Collect Result_Fill2']

In [56]:
train_data['CURE_STANDBY_DISTANCE_Fill2']  = train_data['CURE START POSITION Z Collect Result_Fill2'] - train_data['CURE STANDBY POSITION Z Collect Result_Fill2']
test_data['CURE_STANDBY_DISTANCE_Fill2']  = test_data['CURE START POSITION Z Collect Result_Fill2'] - test_data['CURE STANDBY POSITION Z Collect Result_Fill2']

In [57]:
# 변수 제거
train_data.drop(columns=[
    'CURE END POSITION X Collect Result_Fill2'
    , 'CURE END POSITION Z Collect Result_Fill2'
    , 'CURE START POSITION X Collect Result_Fill2'
    , 'CURE START POSITION Z Collect Result_Fill2'
    , 'CURE_STANDBY_DISTANCE_Fill2'
    , 'CURE SPEED Collect Result_Fill2'], inplace=True)

test_data.drop(columns=[
    'CURE END POSITION X Collect Result_Fill2'
    , 'CURE END POSITION Z Collect Result_Fill2'
    , 'CURE START POSITION X Collect Result_Fill2'
    , 'CURE START POSITION Z Collect Result_Fill2'
    , 'CURE_STANDBY_DISTANCE_Fill2'
    , 'CURE SPEED Collect Result_Fill2'], inplace=True)

In [58]:
# CURE STANDBY POSITION Z Collect Result_Fill2 이름 변경
train_data.rename(columns={'CURE STANDBY POSITION Z Collect Result_Fill2': 'CURE_STANDBY_POSITION_Fill2'}, inplace=True)
test_data.rename(columns={'CURE STANDBY POSITION Z Collect Result_Fill2': 'CURE_STANDBY_POSITION_Fill2'}, inplace=True)

In [59]:
import numpy as np

# 각 스테이지의 좌표 열 정의
stage1_cols = ['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2',
               'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill2',
               'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2']

stage2_cols = ['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2',
               'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2',
               'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2']

stage3_cols = ['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2',
               'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill2',
               'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill2']

# 거리 계산 함수
def calculate_distances(data):
    data['DISTANCE_STAGE1_STAGE2_Fill2'] = np.sqrt(
        (data[stage2_cols[0]] - data[stage1_cols[0]]) ** 2 +
        (data[stage2_cols[1]] - data[stage1_cols[1]]) ** 2 +
        (data[stage2_cols[2]] - data[stage1_cols[2]]) ** 2
    )

    data['DISTANCE_STAGE2_STAGE3_Fill2'] = np.sqrt(
        (data[stage3_cols[0]] - data[stage2_cols[0]]) ** 2 +
        (data[stage3_cols[1]] - data[stage2_cols[1]]) ** 2 +
        (data[stage3_cols[2]] - data[stage2_cols[2]]) ** 2
    )

    data['DISTANCE_STAGE1_STAGE3_Fill2'] = np.sqrt(
        (data[stage3_cols[0]] - data[stage1_cols[0]]) ** 2 +
        (data[stage3_cols[1]] - data[stage1_cols[1]]) ** 2 +
        (data[stage3_cols[2]] - data[stage1_cols[2]]) ** 2
    )

    return data

# train_data에 적용
train_data = calculate_distances(train_data)

# test_data에 적용
test_data = calculate_distances(test_data)

In [60]:
# OK 값이면 1, 결측 값이면 0을 부여
train_data['HEAD_NORMAL_stage1_ok_Fill2'] = train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill2'].apply(lambda x: 1 if x == 'OK' else 0)
test_data['HEAD_NORMAL_stage1_ok_Fill2'] = test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill2'].apply(lambda x: 1 if x == 'OK' else 0)

# HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1 변수 제거
train_data.drop(columns=['HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill2'], inplace=True)
test_data.drop(columns=['HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill2'], inplace=True)

In [61]:
# 변수 제거
train_data.drop(columns=[
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'
    , 'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2'
    , 'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2'
    , 'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill2'
    , 'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2'
    , 'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill2'
    , 'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2'
    , 'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2'
    , 'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill2'], inplace=True)

test_data.drop(columns=[
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'
    , 'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2'
    , 'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2'
    , 'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill2'
    , 'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2'
    , 'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill2'
    , 'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2'
    , 'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2'
    , 'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill2'], inplace=True)

In [62]:
# '_Fill2'를 포함하는 열 이름 필터링
Process_Desc_col = train_data.filter(like='_Fill2').columns

# 필터링된 열 이름 출력
print("<Fill2 공정 관련 변수>")
for col in Process_Desc_col:
    print(col)

<Fill2 공정 관련 변수>
CURE_STANDBY_POSITION_Fill2
Head Purge Position Z Collect Result_Fill2
Machine Tact time Collect Result_Fill2
PalletID Collect Result_Fill2
Production Qty Collect Result_Fill2
Receip No Collect Result_Fill2
WorkMode Collect Result_Fill2
CURE_DISTANCE_Fill2
CURE_Time_Fill2
DISTANCE_STAGE1_STAGE2_Fill2
DISTANCE_STAGE2_STAGE3_Fill2
DISTANCE_STAGE1_STAGE3_Fill2
HEAD_NORMAL_stage1_ok_Fill2


---

In [63]:
train_data.info()
print('---')
# test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Data columns (total 69 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   Model.Suffix                                             40506 non-null  object 
 1   Workorder                                                40506 non-null  object 
 2   DISCHARGED SPEED OF RESIN Collect Result_Dam             40506 non-null  int64  
 3   Head Clean Position Z Collect Result_Dam                 40506 non-null  float64
 4   Head Purge Position Z Collect Result_Dam                 40506 non-null  float64
 5   Head Zero Position Y Collect Result_Dam                  40506 non-null  float64
 6   Head Zero Position Z Collect Result_Dam                  40506 non-null  float64
 7   Machine Tact time Collect Result_Dam                     40506 non-null  float64
 8   PalletID Collect Result_Da

In [64]:
# train_data.info()
print('---')
test_data.info()

---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17361 entries, 0 to 17360
Data columns (total 70 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   Set ID                                                   17361 non-null  object 
 1   Model.Suffix                                             17361 non-null  object 
 2   Workorder                                                17361 non-null  object 
 3   DISCHARGED SPEED OF RESIN Collect Result_Dam             17361 non-null  int64  
 4   Head Clean Position Z Collect Result_Dam                 17361 non-null  float64
 5   Head Purge Position Z Collect Result_Dam                 17361 non-null  float64
 6   Head Zero Position Y Collect Result_Dam                  17361 non-null  float64
 7   Head Zero Position Z Collect Result_Dam                  17361 non-null  float64
 8   Machine Tact time Coll

---

## Modeling

In [65]:
# 각 변수별로 결측값이 존재하는지 확인하는 코드
missing_values = train_data.isnull().sum()

# 결측값이 존재하는 변수와 그 개수 출력
missing_values = missing_values[missing_values > 0]
print(missing_values)

# 결측값이 존재하는 변수명을 리스트에 담기
missing_columns = missing_values.index.tolist()
# print("결측값이 존재하는 변수명:", missing_columns)

WorkMode Collect Result_Dam                                24059
HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1    29213
WorkMode Collect Result_Fill2                              24059
dtype: int64


In [66]:
# 결측값이 존재하는 변수들의 value_counts 계산
for column in missing_columns:
    print(f"Column: {column}")
    print(train_data[column].value_counts())
    print("\n")

Column: WorkMode Collect Result_Dam
WorkMode Collect Result_Dam
7.0    16447
Name: count, dtype: int64


Column: HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1
HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1
OK    11293
Name: count, dtype: int64


Column: WorkMode Collect Result_Fill2
WorkMode Collect Result_Fill2
0.0    16447
Name: count, dtype: int64




In [67]:
# "OK" 값을 가진 변수를 찾고 제거
columns_to_drop = [column for column in missing_columns if train_data[column].apply(lambda x: np.any(x == "OK")).any()]

# 드롭할 변수명 출력 (한 줄에 2개씩)
print("드롭할 변수명:")
for i in range(0, len(columns_to_drop), 2):
    print(columns_to_drop[i:i+2])

# 변수 드롭
train_data = train_data.drop(columns=columns_to_drop)
test_data = test_data.drop(columns=columns_to_drop)

드롭할 변수명:
['HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1']


In [68]:
# 각 변수별로 결측값이 존재하는지 확인하는 코드
missing_values = train_data.isnull().sum()

# 결측값이 존재하는 변수와 그 개수 출력
missing_values = missing_values[missing_values > 0]
print(missing_values)

# 결측값이 존재하는 변수명을 리스트에 담기
missing_columns = missing_values.index.tolist()
# print("결측값이 존재하는 변수명:", missing_columns)

WorkMode Collect Result_Dam      24059
WorkMode Collect Result_Fill2    24059
dtype: int64


In [69]:
# 그룹화할 변수들
groupby_columns = [
    "WorkMode Collect Result_Dam"
    , "WorkMode Collect Result_Fill2"
    ]

# 그룹화하여 target 변수 값의 비율을 계산
grouped = train_data.groupby(groupby_columns)["target"].value_counts(normalize=True).unstack().fillna(0)

# 결과 출력
print(grouped)

target                                                     AbNormal    Normal
WorkMode Collect Result_Dam WorkMode Collect Result_Fill2                    
7.0                         0.0                            0.073326  0.926674


In [70]:
# WorkMode Collect Result_Dam의 이름을 WorkMode Collect Result로 변경
train_data = train_data.rename(columns={'WorkMode Collect Result_Dam': 'WorkMode Collect Result'})
test_data = test_data.rename(columns={'WorkMode Collect Result_Dam': 'WorkMode Collect Result'})

# WorkMode Collect Result_Fill1, WorkMode Collect Result_Fill2 열 드롭
train_data = train_data.drop(columns=['WorkMode Collect Result_Fill1', 'WorkMode Collect Result_Fill2'])
test_data = test_data.drop(columns=['WorkMode Collect Result_Fill1', 'WorkMode Collect Result_Fill2'])

In [71]:
# WorkMode Collect Result 열의 값이 7인 행을 1로 변경
train_data['WorkMode Collect Result'] = train_data['WorkMode Collect Result'].replace(7, 1)
test_data['WorkMode Collect Result'] = test_data['WorkMode Collect Result'].replace(7, 1)

# WorkMode Collect Result 열의 결측값을 0으로 채움
train_data['WorkMode Collect Result'] = train_data['WorkMode Collect Result'].fillna(0)
test_data['WorkMode Collect Result'] = test_data['WorkMode Collect Result'].fillna(0)

In [72]:
test_data['WorkMode Collect Result'].value_counts()

WorkMode Collect Result
0.0    10349
1.0     7012
Name: count, dtype: int64

In [73]:
# 각 변수별로 결측값이 존재하는지 확인하는 코드
missing_values = train_data.isnull().sum()

# 결측값이 존재하는 변수와 그 개수 출력
missing_values = missing_values[missing_values > 0]
print(missing_values)

# 결측값이 존재하는 변수명을 리스트에 담기
missing_columns = missing_values.index.tolist()
print("결측값이 존재하는 변수명:", missing_columns)

Series([], dtype: int64)
결측값이 존재하는 변수명: []


In [74]:
# 각 변수별로 결측값이 존재하는지 확인하는 코드
missing_values = test_data.isnull().sum()

# 결측값이 존재하는 변수와 그 개수 출력
missing_values = missing_values[missing_values > 0]
print(missing_values)

# 결측값이 존재하는 변수명을 리스트에 담기
missing_columns = missing_values.index.tolist()
print("결측값이 존재하는 변수명:", missing_columns)

target    17361
dtype: int64
결측값이 존재하는 변수명: ['target']


### 문자형(object) 변수 -> 수치형 변환


In [75]:
train_data['target'].value_counts()

target
Normal      38156
AbNormal     2350
Name: count, dtype: int64

In [76]:
# 세 변수의 값이 동일하면 해당 값을 가져가고, 하나라도 일치하지 않으면 0의 값을 가지는 파생 변수 생성
train_data['Receip_No_Collect_Result'] = train_data.apply(
    lambda row: row['Receip No Collect Result_Dam'] 
                if (row['Receip No Collect Result_Dam'] == row['Receip No Collect Result_Fill1'] == row['Receip No Collect Result_Fill2']) 
                else 0, 
    axis=1
)

test_data['Receip_No_Collect_Result'] = test_data.apply(
    lambda row: row['Receip No Collect Result_Dam'] 
                if (row['Receip No Collect Result_Dam'] == row['Receip No Collect Result_Fill1'] == row['Receip No Collect Result_Fill2']) 
                else 0, 
    axis=1
)

# 결과 확인
print(train_data[[
    'Receip No Collect Result_Dam'
    , 'Receip No Collect Result_Fill1'
    , 'Receip No Collect Result_Fill2'
    , 'Receip_No_Collect_Result']].value_counts())

Receip No Collect Result_Dam  Receip No Collect Result_Fill1  Receip No Collect Result_Fill2  Receip_No_Collect_Result
1                             1                               1                               1                           39276
6                             6                               6                               6                             980
9                             9                               9                               9                              96
17                            17                              17                              17                             86
3                             3                               3                               3                              64
1                             6                               6                               0                               3
                              4                               1                               0                  

In [77]:
# 결과 확인
print(test_data[[
    'Dispenser_1'
    , 'Dispenser_2']].value_counts())

Dispenser_1  Dispenser_2
1            0              10731
0            1               6618
             0                 12
Name: count, dtype: int64


In [78]:
# 결과 확인
print(test_data[[
    'Receip No Collect Result_Dam'
    , 'Receip No Collect Result_Fill1'
    , 'Receip No Collect Result_Fill2'
    , 'Receip_No_Collect_Result']].value_counts())

Receip No Collect Result_Dam  Receip No Collect Result_Fill1  Receip No Collect Result_Fill2  Receip_No_Collect_Result
1.0                           1.0                             1.0                             1.0                         16829
6.0                           6.0                             6.0                             6.0                           438
9.0                           9.0                             9.0                             9.0                            34
3.0                           3.0                             3.0                             3.0                            31
17.0                          17.0                            17.0                            17.0                           29
Name: count, dtype: int64


In [79]:
# 변수 제거
train_data.drop(columns=[
    'Receip No Collect Result_Dam'
    , 'Receip No Collect Result_Fill1'
    , 'Receip No Collect Result_Fill2'], inplace=True)

test_data.drop(columns=[
    'Receip No Collect Result_Dam'
    , 'Receip No Collect Result_Fill1'
    , 'Receip No Collect Result_Fill2'], inplace=True)

In [80]:
train_data['Receip_No_Collect_Result'].value_counts()

Receip_No_Collect_Result
1     39276
6       980
9        96
17       86
3        64
0         4
Name: count, dtype: int64

In [81]:
# 세 변수의 값이 동일하면 해당 값을 가져가고, 하나라도 일치하지 않으면 0의 값을 가지는 파생 변수 생성
train_data['PalletID_Collect_Result'] = train_data.apply(
    lambda row: row['PalletID Collect Result_Dam'] 
                if (row['PalletID Collect Result_Dam'] == row['PalletID Collect Result_Fill1'] == row['PalletID Collect Result_Fill2']) 
                else 0, 
    axis=1
)

test_data['PalletID_Collect_Result'] = test_data.apply(
    lambda row: row['PalletID Collect Result_Dam'] 
                if (row['PalletID Collect Result_Dam'] == row['PalletID Collect Result_Fill1'] == row['PalletID Collect Result_Fill2']) 
                else 0, 
    axis=1
)

# 결과 확인
print(train_data[[
    'PalletID Collect Result_Dam'
    , 'PalletID Collect Result_Fill1'
    , 'PalletID Collect Result_Fill2']].value_counts())

PalletID Collect Result_Dam  PalletID Collect Result_Fill1  PalletID Collect Result_Fill2
4                            4                              4                                3174
3                            3                              3                                3166
1                            1                              1                                3146
2                            2                              2                                3126
6                            6                              6                                3114
                                                                                             ... 
                             12                             12                                  1
                             9                              9                                   1
                             6                              8                                   1
2                           

In [82]:
# 변수 제거
train_data.drop(columns=[
    'PalletID Collect Result_Dam'
    , 'PalletID Collect Result_Fill1'
    , 'PalletID Collect Result_Fill2'], inplace=True)

test_data.drop(columns=[
    'PalletID Collect Result_Dam'
    , 'PalletID Collect Result_Fill1'
    , 'PalletID Collect Result_Fill2'], inplace=True)

In [83]:
# 세 변수의 값이 동일하면 해당 값을 가져가고, 하나라도 일치하지 않으면 0의 값을 가지는 파생 변수 생성
train_data['Production_Qty_Collect_Result'] = train_data.apply(
    lambda row: row['Production Qty Collect Result_Dam'] 
                if (row['Production Qty Collect Result_Dam'] == row['Production Qty Collect Result_Fill1'] == row['Production Qty Collect Result_Fill2']) 
                else 0, 
    axis=1
)

test_data['Production_Qty_Collect_Result'] = test_data.apply(
    lambda row: row['Production Qty Collect Result_Dam'] 
                if (row['Production Qty Collect Result_Dam'] == row['Production Qty Collect Result_Fill1'] == row['Production Qty Collect Result_Fill2']) 
                else 0, 
    axis=1
)

In [84]:
# 변수 제거
train_data.drop(columns=[
    'Production Qty Collect Result_Dam'
    , 'Production Qty Collect Result_Fill1'
    , 'Production Qty Collect Result_Fill2'], inplace=True)

test_data.drop(columns=[
    'Production Qty Collect Result_Dam'
    , 'Production Qty Collect Result_Fill1'
    , 'Production Qty Collect Result_Fill2'], inplace=True)

In [85]:
print(test_data['target'].dtype)

float64


In [86]:
import pandas as pd

def calculate_total_time_and_ratios(data):
    data['total_time'] = (
        data['Machine Tact time Collect Result_Dam'] +
        data['Machine Tact time Collect Result_Fill1'] +
        data['Machine Tact time Collect Result_Fill2'] +
        data['Chamber Temp. Unit Time_AutoClave']
    )
    data['time_ratio_Dam'] = (data['Machine Tact time Collect Result_Dam'] / data['total_time']).round(3)
    data['time_ratio_Fill1'] = (data['Machine Tact time Collect Result_Fill1'] / data['total_time']).round(3)
    data['time_ratio_Fill2'] = (data['Machine Tact time Collect Result_Fill2'] / data['total_time']).round(3)
    data['time_ratio_AutoClave'] = (data['Chamber Temp. Unit Time_AutoClave'] / data['total_time']).round(3)
    return data

# train_data와 test_data에 함수 적용
train_data = calculate_total_time_and_ratios(train_data)
test_data = calculate_total_time_and_ratios(test_data)

In [87]:
# 변수 제거
train_data.drop(columns=[
    'total_time'
    , 'Machine Tact time Collect Result_Dam'
    , 'Machine Tact time Collect Result_Fill1'
    , 'Machine Tact time Collect Result_Fill2'
    , 'Chamber Temp. Unit Time_AutoClave'], inplace=True)

test_data.drop(columns=[
    'total_time'
    , 'Machine Tact time Collect Result_Dam'
    , 'Machine Tact time Collect Result_Fill1'
    , 'Machine Tact time Collect Result_Fill2'
    , 'Chamber Temp. Unit Time_AutoClave'], inplace=True)

In [88]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Data columns (total 60 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Model.Suffix                                           40506 non-null  object 
 1   Workorder                                              40506 non-null  object 
 2   DISCHARGED SPEED OF RESIN Collect Result_Dam           40506 non-null  int64  
 3   Head Clean Position Z Collect Result_Dam               40506 non-null  float64
 4   Head Purge Position Z Collect Result_Dam               40506 non-null  float64
 5   Head Zero Position Y Collect Result_Dam                40506 non-null  float64
 6   Head Zero Position Z Collect Result_Dam                40506 non-null  float64
 7   WorkMode Collect Result                                40506 non-null  float64
 8   1st Pressure Collect Result_AutoClave         

In [89]:
train_data['Chamber_Temp_ok_AutoClave'].value_counts()

Chamber_Temp_ok_AutoClave
1    29112
0    11394
Name: count, dtype: int64

In [90]:
train_data['HEAD_NORMAL_stage1_ok_Fill2'].value_counts()

HEAD_NORMAL_stage1_ok_Fill2
0    29213
1    11293
Name: count, dtype: int64

In [91]:
# 변수 제거
train_data.drop(columns=[
    'Chamber_Temp_ok_AutoClave'
    , 'HEAD_NORMAL_stage1_ok_Fill2'], inplace=True)

test_data.drop(columns=[
    'Chamber_Temp_ok_AutoClave'
    , 'HEAD_NORMAL_stage1_ok_Fill2'], inplace=True)

In [92]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Data columns (total 58 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Model.Suffix                                           40506 non-null  object 
 1   Workorder                                              40506 non-null  object 
 2   DISCHARGED SPEED OF RESIN Collect Result_Dam           40506 non-null  int64  
 3   Head Clean Position Z Collect Result_Dam               40506 non-null  float64
 4   Head Purge Position Z Collect Result_Dam               40506 non-null  float64
 5   Head Zero Position Y Collect Result_Dam                40506 non-null  float64
 6   Head Zero Position Z Collect Result_Dam                40506 non-null  float64
 7   WorkMode Collect Result                                40506 non-null  float64
 8   1st Pressure Collect Result_AutoClave         

### 타겟 인코딩

In [93]:
# 'target' 열의 변수 타입을 object로 변경
# -> test 데이터는 float64 타입으로 되어있음 
test_data['target'] = test_data['target'].astype('object')

# object 타입의 변수 출력
train_object_columns = train_data.select_dtypes(include=['object']).columns
test_object_columns = test_data.select_dtypes(include=['object']).columns

print(train_object_columns, f" train_object_columns 갯수 : {len(train_object_columns)}")
print(test_object_columns, f" test_object_columns 갯수 : {len(test_object_columns)}")

# 각 object 변수의 고유 값 개수 출력
print("\nTrain Data:")
for col in train_object_columns:
    unique_count = train_data[col].nunique()
    print(f"{col} unique 값 갯수: {unique_count}")

print("\nTest Data:")
for col in test_object_columns:
    unique_count = test_data[col].nunique()
    print(f"{col} unique 값 갯수: {unique_count}")

Index(['Model.Suffix', 'Workorder', 'target'], dtype='object')  train_object_columns 갯수 : 3
Index(['Set ID', 'Model.Suffix', 'Workorder', 'target'], dtype='object')  test_object_columns 갯수 : 4

Train Data:
Model.Suffix unique 값 갯수: 7
Workorder unique 값 갯수: 663
target unique 값 갯수: 2

Test Data:
Set ID unique 값 갯수: 17361
Model.Suffix unique 값 갯수: 7
Workorder unique 값 갯수: 662
target unique 값 갯수: 0


In [94]:
# 필요한 라이브러리 임포트
import pandas as pd
import category_encoders as ce

# 타겟 변수와 범주형 변수 지정
## Target Encoding의 smoothing 파라미터는 default로 auto로 설정되어 있음
target = 'target'  # 타겟 변수 이름으로 변경
categorical_columns = [
    'Model.Suffix',
    'Workorder',
]  # 범주형 변수 이름으로 변경

# 타겟 값을 숫자로 변환
target_mapping = {'Normal': 0, 'AbNormal': 1}
train_data[target] = train_data[target].map(target_mapping)
test_data[target] = test_data[target].map(target_mapping)

# 열이 존재하는지 확인
missing_columns = [col for col in categorical_columns if col not in train_data.columns]
if missing_columns:
    raise ValueError(f"train_data에 다음 열이 존재하지 않습니다: {missing_columns}")

# 타겟 인코더 생성 및 학습
encoder = ce.TargetEncoder(cols=categorical_columns)
train_data = encoder.fit_transform(train_data, train_data[target])

# Set ID 열을 별도로 저장
set_id = test_data['Set ID']

# 테스트 데이터 인코딩 (Set ID 열 제외)
test_data = test_data.drop(columns=['Set ID'])
test_data = encoder.transform(test_data)

# Set ID 열을 맨 앞에 추가
test_data.insert(0, 'Set ID', set_id)

# categorical_columns에 해당하는 열의 데이터 값만 확인
print(train_data[categorical_columns].head(3))
print(test_data[categorical_columns].head(3))

# 역 매핑 딕셔너리 생성
reverse_target_mapping = {v: k for k, v in target_mapping.items()}

# 타겟 값을 원래대로 변환
train_data[target] = train_data[target].map(reverse_target_mapping)
test_data[target] = test_data[target].map(reverse_target_mapping)

print("--- train_data ---")

# 변환된 타겟 값 확인
print(train_data[[target]].value_counts())

   Model.Suffix  Workorder
0      0.049336   0.158385
1      0.049336   0.015314
2      0.056712   0.009534
   Model.Suffix  Workorder
0      0.056712   0.091912
1      0.056712   0.024247
2      0.056712   0.091463
--- train_data ---
target  
Normal      38156
AbNormal     2350
Name: count, dtype: int64


In [95]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Data columns (total 58 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Model.Suffix                                           40506 non-null  float64
 1   Workorder                                              40506 non-null  float64
 2   DISCHARGED SPEED OF RESIN Collect Result_Dam           40506 non-null  int64  
 3   Head Clean Position Z Collect Result_Dam               40506 non-null  float64
 4   Head Purge Position Z Collect Result_Dam               40506 non-null  float64
 5   Head Zero Position Y Collect Result_Dam                40506 non-null  float64
 6   Head Zero Position Z Collect Result_Dam                40506 non-null  float64
 7   WorkMode Collect Result                                40506 non-null  float64
 8   1st Pressure Collect Result_AutoClave         

In [96]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17361 entries, 0 to 17360
Data columns (total 59 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Set ID                                                 17361 non-null  object 
 1   Model.Suffix                                           17361 non-null  float64
 2   Workorder                                              17361 non-null  float64
 3   DISCHARGED SPEED OF RESIN Collect Result_Dam           17361 non-null  int64  
 4   Head Clean Position Z Collect Result_Dam               17361 non-null  float64
 5   Head Purge Position Z Collect Result_Dam               17361 non-null  float64
 6   Head Zero Position Y Collect Result_Dam                17361 non-null  float64
 7   Head Zero Position Z Collect Result_Dam                17361 non-null  float64
 8   WorkMode Collect Result                       

### 데이터 분할

In [97]:
# 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    train_data.drop("target", axis=1),
    train_data["target"],
    test_size=0.2,
    shuffle=True,
    random_state=RANDOM_STATE,
)

In [98]:
x_val

,Model.Suffix,Workorder,DISCHARGED SPEED OF RESIN Collect Result_Dam,Head Clean Position Z Collect Result_Dam,Head Purge Position Z Collect Result_Dam,Head Zero Position Y Collect Result_Dam,Head Zero Position Z Collect Result_Dam,WorkMode Collect Result,1st Pressure Collect Result_AutoClave,2nd Pressure Collect Result_AutoClave,...,DISTANCE_STAGE1_STAGE2_Fill2,DISTANCE_STAGE2_STAGE3_Fill2,DISTANCE_STAGE1_STAGE3_Fill2,Receip_No_Collect_Result,PalletID_Collect_Result,Production_Qty_Collect_Result,time_ratio_Dam,time_ratio_Fill1,time_ratio_Fill2,time_ratio_AutoClave
40143,0.056712,0.044053,10,130.85,130.85,300.0,265.00,0.0,0.303,0.305,...,377.500013,302.000017,679.5,1,1,258,0.111,0.111,0.030,0.748
14916,0.056712,0.098997,10,133.50,133.50,303.5,265.00,1.0,0.305,0.306,...,377.500013,302.000017,679.5,1,2,411,0.129,0.128,0.027,0.716
35683,0.056712,0.039025,16,124.00,130.85,300.0,265.02,0.0,0.311,0.491,...,377.500013,302.000017,679.5,1,3,98,0.144,0.112,0.038,0.707
15843,0.049336,0.020833,16,124.00,130.85,300.0,265.00,0.0,0.299,0.492,...,377.500013,302.000017,679.5,1,8,108,0.139,0.113,0.039,0.710
6612,0.056712,0.057692,10,133.50,133.50,303.5,265.00,1.0,0.313,0.497,...,377.500013,302.000017,679.5,1,1,358,0.137,0.134,0.028,0.702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,0.049336,0.158385,16,124.00,130.85,300.0,265.00,0.0,0.310,0.491,...,195.000000,193.000000,388.0,1,13,82,0.123,0.114,0.040,0.723
28872,0.056712,0.036113,16,124.00,130.85,300.0,265.00,0.0,0.313,0.492,...,377.500013,302.000017,679.5,1,1,169,0.138,0.112,0.040,0.709
38867,0.056712,0.059686,10,133.50,133.50,300.0,265.00,1.0,0.315,0.317,...,377.500013,302.000017,679.5,1,6,44,0.130,0.133,0.029,0.708
35657,0.056712,0.000288,16,130.85,130.85,300.0,265.02,0.0,0.304,0.490,...,377.500013,302.000017,679.5,1,3,83,0.111,0.112,0.040,0.737


In [99]:
df_train, df_val = train_test_split(
    train_data,
    test_size=0.2,
    stratify=train_data["target"],
    random_state=RANDOM_STATE,
)

def print_stats(df: pd.DataFrame):
    num_normal = len(df[df["target"] == "Normal"])
    num_abnormal = len(df[df["target"] == "AbNormal"])

    print(f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}" + f" ratio: {num_abnormal/num_normal}")


# Print statistics
print(f"  \tAbnormal\tNormal")
print_stats(df_train)
print_stats(df_val)

  	Abnormal	Normal
  Total: Normal: 30524, AbNormal: 1880 ratio: 0.06159087930808544
  Total: Normal: 7632, AbNormal: 470 ratio: 0.061582809224318656


## 3. 모델 학습

### 모델 정의

In [100]:
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

def get_clf_eval(y_test, y_pred_proba, threshold=0.5):
    # 확률을 기준으로 예측 레이블 생성
    y_pred = (y_pred_proba >= threshold).astype(int)  # 0.5 이상의 확률을 양성으로 간주

    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print("Confusion Matrix:\n", confusion)
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")


In [101]:
from lightgbm import LGBMClassifier

optuna

In [102]:
# import optuna
# from lightgbm import LGBMClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import f1_score

# # 'Normal'과 'AbNormal'을 숫자로 변환
# train_data['target'] = train_data['target'].map({'Normal': 0, 'AbNormal': 1})

# def objectiveLGBM_dart(trial, x_tr, y_tr, x_val, y_val):
#     param = {
#         'n_estimators': trial.suggest_int('n_estimators', 300, 5000),
#         'num_leaves': trial.suggest_int('num_leaves', 300, 4000),
#         'max_depth': trial.suggest_int('max_depth', 10, 300),
#         'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
#         'min_child_samples': trial.suggest_int('min_child_samples', 3, 300),
        
#         'boosting': 'dart',  # dart 사용
#         'random_state': RANDOM_STATE,
#         'verbose': -1
#     }
       
#     model = LGBMClassifier(**param)
#     model.fit(x_tr, y_tr)
#     pred = model.predict(x_val)
#     score = f1_score(y_val, pred, average="binary")
    
#     return score

# # 데이터셋 분할
# x_train, x_val, y_train, y_val = train_test_split(
#     train_data.drop("target", axis=1),
#     train_data["target"],
#     test_size=0.2,
#     shuffle=True,
#     random_state=RANDOM_STATE,
# )

# # 하이퍼 파라미터 튜닝
# study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
# study.optimize(lambda trial: objectiveLGBM_dart(trial, x_train, y_train, x_val, y_val), n_trials=3000)

# print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

200번까지 했을때 400분 정도 걸렸고 이때 튜닝과정 끊었음(시간상의 이유로)

{'n_estimators': 1019, 'num_leaves': 1513, 'max_depth': 115, 'learning_rate': 0.   010329666890588058, 'min_child_samples': 7}. Best is trial 129 with value: 0.22495606326889278.

In [103]:
# LightGBM
lgb_model = LGBMClassifier(
    n_estimators=1019
    , num_leaves=1513
    , max_depth=115
    , learning_rate=0.010329666890588058
    , min_child_samples=7
    , verbose = -1
    , boosting= 'dart'
    , random_state=RANDOM_STATE
)

### 모델 학습


In [104]:
# df_train 데이터로 학습
df_train["target"] = df_train["target"].apply(lambda x: 1 if x == 'AbNormal' else 0)

# features = df_train.columns.tolist()

train_x = df_train.drop(columns=["target"])
train_y = df_train["target"]

lgb_model.fit(train_x, train_y)

LGBMClassifier(boosting='dart', learning_rate=0.010329666890588058,
               max_depth=115, min_child_samples=7, n_estimators=1019,
               num_leaves=1513, random_state=110, verbose=-1)

In [105]:
# 검증 데이터 준비
df_val["target"] = df_val["target"].apply(lambda x: 1 if x == 'AbNormal' else 0)

val_x = df_val.drop(columns=["target"])
val_y = df_val["target"]

# 확률 예측
y_pred_proba = lgb_model.predict_proba(val_x)[:, 1]  # 양성 클래스에 대한 확률

# 모델 성능 평가
get_clf_eval(val_y, y_pred_proba)


Confusion Matrix:
 [[7565   67]
 [ 410   60]]
Accuracy: 0.9411
Precision: 0.4724
Recall: 0.1277
F1 Score: 0.2010


분할한 데이터 -> 원래의 데이터(train data)로 학습한 새 모델 생성

In [106]:
# df_train 데이터로 학습
train_data["target"] = train_data["target"].apply(lambda x: 1 if x == 'AbNormal' else 0)

train_x_all = train_data.drop(columns=["target"])
train_y_all = train_data["target"]

lgb_model.fit(train_x_all, train_y_all)

LGBMClassifier(boosting='dart', learning_rate=0.010329666890588058,
               max_depth=115, min_child_samples=7, n_estimators=1019,
               num_leaves=1513, random_state=110, verbose=-1)

In [107]:
# 예측에 필요한 데이터 분리
x_test = test_data.drop(["target", "Set ID"], axis=1)

# 테스트 데이터에 대한 예측 확률
soft_voting_probs = lgb_model.predict_proba(x_test)[:, 1]

# 스레숄드 조절
threshold = 0.5 #default
soft_voting_preds = [1 if prob >= threshold else 0 for prob in soft_voting_probs]

# 테스트 데이터에서 AbNormal로 예측된 개수 출력
print(sum(soft_voting_preds))

224


Public Score : 0.1824  

In [108]:
# 예측에 필요한 데이터 분리
x_test = test_data.drop(["target", "Set ID"], axis=1)

# 테스트 데이터에 대한 예측 확률
soft_voting_probs = lgb_model.predict_proba(x_test)[:, 1]

# 스레스홀드 조절
threshold = 0.2
soft_voting_preds = [1 if prob >= threshold else 0 for prob in soft_voting_probs]

# 테스트 데이터에서 AbNormal로 예측된 개수 출력
print(sum(soft_voting_preds))

1115


Public Score : 0.22095238095238093 

In [109]:
# 예측에 필요한 데이터 분리
x_test = test_data.drop(["target", "Set ID"], axis=1)

# 테스트 데이터에 대한 예측 확률
soft_voting_probs = lgb_model.predict_proba(x_test)[:, 1]

# 스레스홀드 조절
threshold = 0.15
soft_voting_preds = [1 if prob >= threshold else 0 for prob in soft_voting_probs]

# 테스트 데이터에서 AbNormal로 예측된 개수 출력
print(sum(soft_voting_preds))

1592


Public Score : 0.2152886115444618

In [110]:
# 예측에 필요한 데이터 분리
x_test = test_data.drop(["target", "Set ID"], axis=1)

# 테스트 데이터에 대한 예측 확률
soft_voting_probs = lgb_model.predict_proba(x_test)[:, 1]

# 스레스홀드 조절
threshold = 0.1
soft_voting_preds = [1 if prob >= threshold else 0 for prob in soft_voting_probs]

# 테스트 데이터에서 AbNormal로 예측된 개수 출력
print(sum(soft_voting_preds))

2671


Public Score : 0.18926397343663529

## 4. 제출하기


### 제출 파일 작성


In [111]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("C:/Users/KimDongyoung/Desktop/git_LGaimers5/Lg_aimers5/data/submission.csv")
df_sub["target"] = soft_voting_preds

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

In [112]:
df_sub['target'].value_counts()

target
Normal      14690
AbNormal     2671
Name: count, dtype: int64

In [113]:
df_sub.head(10)

,Set ID,target
0,0001be084fbc4aaa9d921f39e595961b,Normal
1,0005bbd180064abd99e63f9ed3e1ac80,Normal
2,000948934c4140d883d670adcb609584,Normal
3,000a6bfd02874c6296dc7b2e9c5678a7,Normal
4,0018e78ce91343678716e2ea27a51c95,Normal
5,001fda4596f545d0a3b0ce85fbea77d2,Normal
6,0020734a7b29472298358ad58645a0c9,Normal
7,00234c5914cd4c4a888d13f8b3773135,Normal
8,00297b6c93e44d49ac534758a23dc74e,Normal
9,002d904240d84b188d410d16383a9c3a,Normal


**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**
